# Exploratory Data Analysis

In [7]:
import pandas as pd
import numpy as np
import math, requests, json, re, time, sys, ast, warnings
from sklearn.preprocessing import *

warnings.filterwarnings('ignore')

## Import Data

In [8]:
df = pd.read_csv('datasets/steam_dataset/steamapi/csv/steamapp_api.csv',sep=',')
df.head(2)

,appid,developer,publisher,score_rank,positive,negative,initialprice,discount,ccu,languages,...,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,release_date,metacritic
0,10,Valve,Valve,NaN,190150,4865,999,0,14139,"English, French, German, Italian, Spanish - Sp...",...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,https://cdn.akamai.steamstatic.com/steam/apps/...,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'coming_soon': False, 'date': '1 Nov, 2000'}","{'score': 88, 'url': 'https://www.metacritic.c..."
1,10090,Treyarch,Activision,NaN,35995,2877,1999,0,519,"English, French, Italian, German, Spanish - Spain",...,"Call of Duty is back, redefining war like you'...","Call of Duty is back, redefining war like you'...","Call of Duty is back, redefining war like you'...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.callofduty.com,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...",[],[],"{'coming_soon': False, 'date': '18 Nov, 2008'}","{'score': 83, 'url': 'https://www.metacritic.c..."


In [9]:
df.shape

(39831, 25)

### Data Preprocessing

In [10]:
df_copy = df.copy()
df_copy.reset_index(inplace=True, drop=True)
for i in range(len(df_copy)):
    df_copy['release_date'][i] = ast.literal_eval(df_copy['release_date'][i])
    df_copy['tags'][i] = ast.literal_eval(df_copy['tags'][i])
    df_copy['metacritic'][i] = ast.literal_eval(df_copy['metacritic'][i])

In [11]:
df_copy.fillna(np.NaN, inplace=True)

In [12]:
df_copy['date'] = np.nan
for i in range(len(df_copy)):
    df_copy['date'][i] = list(df_copy['release_date'][i].values())[1]

Splitting Tags and Metacritic

In [13]:
df_copy['steamspy_tags'] = np.nan
df_copy['meta_score'] = np.nan
for i in range(len(df_copy)):
    df_copy['steamspy_tags'][i] = list(df_copy['tags'][i])
    if type(df_copy['metacritic'][i]) == dict:
        df_copy['meta_score'][i] = list(df_copy['metacritic'][i].values())[0]
    else:
        df_copy['meta_score'][i] = 0

In [14]:
df_copy['steamspy_tags'] = df_copy['steamspy_tags'].apply(lambda x: ', '.join(map(str, x)))

Requirements

In [15]:
# df_copy['pc_requirements'] = df_copy['pc_requirements'].map({'[]': np.NaN})

In [16]:
# df_copy[df_copy['pc_requirements'] == '[]']['pc_requirements'] = ''
# df_copy[df_copy['mac_requirements'] == '[]']['mac_requirements'] = ''
# df_copy[df_copy['linux_requirements'] == '[]']['linux_requirements'] = ''

In [17]:
df_pcr = (df['pc_requirements'].str.replace(r'\\[rtn]', '')
                                .str.replace(r'<[pbr]{1,2}>', ' ')
                                .str.replace(r'<[\/"=\w\s]+>', '')
                                )
df_pcr = df_pcr.apply(lambda x: ast.literal_eval(x))

In [18]:
type(df_pcr[0]['minimum'].replace('Minimum:', '').strip())

str

In [19]:
xs = '[]'
xs = xs.replace('[]', '{"Minimum": "-", "Recommended": "-"}')
xs_a = ast.literal_eval(xs)
print(type(xs_a))
print(xs)

<class 'dict'>
{"Minimum": "-", "Recommended": "-"}


In [20]:
df['pc_requirements'].apply(lambda x: x.replace('[]', '{"Minimum": "-", "Recommended": "-"}') 
                                                        if '[]' in x else x)

0        {'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...
1        {'minimum': '<ul class="bb_ul"><li><strong>Sup...
2        {'minimum': '<ul class="bb_ul"><li><strong>OS:...
3        {'minimum': '<strong>Minimum:</strong><br>\t\t...
4        {'minimum': '<strong>Minimum:</strong><br><ul ...
                               ...                        
39826    {'minimum': '<strong>Minimum:</strong><br><ul ...
39827    {'minimum': '<strong>Minimum:</strong><br><ul ...
39828    {'minimum': '<strong>Minimum:</strong><br><ul ...
39829    {'minimum': '<strong>Minimum:</strong><br><ul ...
39830    {'minimum': '<strong>Minimum:</strong><br><ul ...
Name: pc_requirements, Length: 39831, dtype: object

In [21]:
def process_req(df, col):
    df[f'{col}_cleaned'] = df[col].apply(lambda x: x.replace('[]', '{"Minimum": "-", "Recommended": "-"}') 
                                                        if '[]' in x else x)
    df[f'{col}_cleaned'] = (df[f'{col}_cleaned']
                                .str.replace(r'\\[rtn]', '')
                                .str.replace(r'<[pbr]{1,2}>', ' ')
                                .str.replace(r'<[\/"=\w\s]+>', ''))
    df[f'{col}_cleaned'] = df[f'{col}_cleaned'].apply(lambda x: ast.literal_eval(x))
    df[f'{col}_minimum'] = df[f'{col}_cleaned'].apply(lambda x: x['minimum'].replace('Minimum:', '').strip() 
                                                        if 'minimum' in x.keys() else np.nan)
    df[f'{col}_recommended'] = df[f'{col}_cleaned'].apply(lambda x: x['recommended'].replace('Recommended:', '').strip() 
                                                        if 'recommended' in x.keys() else np.nan)
    df.drop(f'{col}_cleaned', axis=1, inplace=True)

    return df

In [22]:
df_copy = process_req(df_copy, 'pc_requirements')
df_copy = process_req(df_copy, 'mac_requirements')
df_copy = process_req(df_copy, 'linux_requirements')

In [23]:
df_copy.head(3)

,appid,developer,publisher,score_rank,positive,negative,initialprice,discount,ccu,languages,...,metacritic,date,steamspy_tags,meta_score,pc_requirements_minimum,pc_requirements_recommended,mac_requirements_minimum,mac_requirements_recommended,linux_requirements_minimum,linux_requirements_recommended
0,10,Valve,Valve,NaN,190150,4865,999,0,14139,"English, French, German, Italian, Spanish - Sp...",...,"{'score': 88, 'url': 'https://www.metacritic.c...","1 Nov, 2000","Action, FPS, Multiplayer, Shooter, Classic, Te...",88.0,"500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN,"Linux Ubuntu 12.04, Dual-core from Intel or AM...",NaN
1,10090,Treyarch,Activision,NaN,35995,2877,1999,0,519,"English, French, Italian, German, Spanish - Spain",...,"{'score': 83, 'url': 'https://www.metacritic.c...","18 Nov, 2008","Zombies, World War II, FPS, Multiplayer, Actio...",83.0,Supported OS: Windows XP/Vista/7 ...,NaN,NaN,NaN,NaN,NaN
2,10180,Infinity Ward,Activision,NaN,41509,3210,1999,0,510,"English, French, German, Italian, Spanish - Spain",...,"{'score': 86, 'url': 'https://www.metacritic.c...","11 Nov, 2009","Action, FPS, Multiplayer, Shooter, First-Perso...",86.0,OS: Microsoft Windows XP or Windows Vista (Win...,NaN,Operating System: 10.12 (Sierra) CPU Process...,NaN,NaN,NaN


In [24]:
df_copy.columns

Index(['appid', 'developer', 'publisher', 'score_rank', 'positive', 'negative',
       'initialprice', 'discount', 'ccu', 'languages', 'genre', 'tags', 'name',
       'required_age', 'is_free', 'detailed_description', 'about_the_game',
       'short_description', 'header_image', 'website', 'pc_requirements',
       'mac_requirements', 'linux_requirements', 'release_date', 'metacritic',
       'date', 'steamspy_tags', 'meta_score', 'pc_requirements_minimum',
       'pc_requirements_recommended', 'mac_requirements_minimum',
       'mac_requirements_recommended', 'linux_requirements_minimum',
       'linux_requirements_recommended'],
      dtype='object')

In [25]:
df_steam = df_copy[['appid', 'name', 'developer', 'publisher',
                    'genre', 'tags', 'is_free', 'initialprice', 'discount', 'ccu',
                    'required_age', 'detailed_description', 'about_the_game',
                    'short_description', 'date', 'pc_requirements_minimum',
                    'pc_requirements_recommended', 'mac_requirements_minimum',
                    'mac_requirements_recommended', 'linux_requirements_minimum',
                    'linux_requirements_recommended', 'steamspy_tags', 'meta_score', 'score_rank',
                    'header_image', 'website', 'languages']]

In [26]:
df_steam.to_csv('D:/Projects/Github/games-requirements-checker/datasets/steam_dataset/steamapi/csv/steam_dataset.csv', index=False)

## Preprocessed Data

In [38]:
import pandas as pd
import numpy as np
import math, requests, json, re, time, sys, ast, warnings, dateparser, locale
from sklearn.preprocessing import *

warnings.filterwarnings('ignore')

In [37]:
df_steam = pd.read_csv('D:/Projects/Github/games-requirements-checker/datasets/steam_dataset/steamapi/csv/steam_dataset.csv', sep=',')

In [45]:
locale.setlocale(locale.LC_ALL, 'en')

df_steam['date'] = pd.to_datetime(df_steam['date'], errors='coerce')
df_steam['date'].dt.strftime('%Y-%m-%d')

0        2000-11-01
1        2008-11-18
2        2009-11-11
3               NaN
4        2019-02-13
            ...    
39826    2019-03-29
39827    2019-07-21
39828    2019-05-30
39829    2021-10-27
39830    2019-03-14
Name: date, Length: 39831, dtype: object

In [28]:
df_steam.head(2)

,appid,name,developer,publisher,genre,tags,is_free,initialprice,discount,ccu,...,mac_requirements_minimum,mac_requirements_recommended,linux_requirements_minimum,linux_requirements_recommended,steamspy_tags,meta_score,score_rank,header_image,website,languages
0,10,Counter-Strike,Valve,Valve,Action,"{'Action': 5374, 'FPS': 4797, 'Multiplayer': 3...",False,999,0,14139,...,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN,"Linux Ubuntu 12.04, Dual-core from Intel or AM...",NaN,"Action, FPS, Multiplayer, Shooter, Classic, Te...",88.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,0,"English, French, German, Italian, Spanish - Sp..."
1,10090,Call of Duty: World at War,Treyarch,Activision,Action,"{'Zombies': 1108, 'World War II': 818, 'FPS': ...",False,1999,0,519,...,NaN,NaN,NaN,NaN,"Zombies, World War II, FPS, Multiplayer, Actio...",83.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.callofduty.com,"English, French, Italian, German, Spanish - Spain"


In [ ]:
df_steam[df_steam['date'] > '2021-10-01']

In [47]:
df_steam.to_csv('D:/Projects/Github/games-requirements-checker/datasets/steam_dataset/steamapi/csv/steam_dataset.csv', index=False)